In [1]:
import pandas as pd
from datetime import datetime

def read_file(file_name, ticker):
    """
    example
    file_name = '//2023-05-20 20_37_07_av_company_financial_data.xlsx'
    ticker = 'GS'
    """
    df_is = pd.read_excel(file_name, sheet_name = ticker+'_INCOME_STATEMENT')
    df_bs = pd.read_excel(file_name, sheet_name = ticker+'_BALANCE_SHEET')
    df_cf = pd.read_excel(file_name, sheet_name = ticker+'_CASH_FLOW')
    df_all = pd.merge(df_is, df_bs, on='fiscalDateEnding', how='outer').merge(df_cf, on='fiscalDateEnding', how='outer')
    return df_all

def calc_ratio(df_all):
    try:
        df_all['ROE'] = df_all.apply(lambda row: row['netIncome_x']/row['totalShareholderEquity'], axis = 1)*4
    except:
        pass
    try:
        df_all['ROA'] = df_all.apply(lambda row: row['netIncome_x']/row['totalAssets'], axis = 1)*4
    except:
        pass
    try:
        df_all['NIM'] = df_all.apply(lambda row: row['netInterestIncome']/row['totalAssets'], axis = 1)*4
    except:
        pass
    df_ratio = df_all[['fiscalDateEnding', 'ROE', 'ROA',  'NIM']]
    return df_ratio

def cal_ratio_2(df):# cal ratio
    try:
        df['NPL'] = df['BHCKS449']/df['BHCK2122']
    except:
        df['NPL'] = 0
    try:
        df['CAR'] = df['BHCA3792']/df['BHCAA223']
    except:
        df['CAR'] = 0
    df_ratio_2 = df[['RSSD9001','RSSD9999','NPL','CAR']]
    return df_ratio_2

In [2]:
file_name = 'data/2023-05-20 20_37_07_av_company_financial_data.xlsx'
file_name_2 = 'data/BHCF20221231.txt'
ticker_list = ['JPM', 'GS', 'MS']
RSSD_list = [1039502,2380443,2162966]
keep_col = ['RSSD9001','RSSD9999','BHCK2122', 'BHCK3123', 'BHCKS449', 'BHCA3792', 'BHCAA223']

# calc ROE, ROA, NIM
df_ratio = pd.DataFrame()

for ticker in ticker_list:
    df_all = read_file(file_name, ticker)
    df_temp = calc_ratio(df_all)
    df_temp['Ticker'] = ticker
    df_ratio = pd.concat([df_ratio, df_temp])
    
# calc NPL, CAR
df = pd.read_csv(file_name_2, sep = '^',low_memory=False)[keep_col]
df = df[df['RSSD9001'].isin(RSSD_list)]
ticker_RSSD_mapping = pd.DataFrame({'Ticker': ticker_list, 'RSSD': RSSD_list})
df_ratio_2 = cal_ratio_2(df)

# convert RSSD to ticker
df_ratio_2 = df_ratio_2.merge(ticker_RSSD_mapping, left_on = 'RSSD9001', right_on = 'RSSD')
df_ratio_2 = df_ratio_2[['RSSD9999','Ticker','NPL','CAR']]
df_ratio_2 = df_ratio_2.rename(columns = {'RSSD9999':'fiscalDateEnding'})

# combine the two dataframe for ratios
df_ratio_2['fiscalDateEnding'] = pd.to_datetime(df_ratio_2['fiscalDateEnding'].astype('str'), format = '%Y-%m-%d').astype('str')
df_ratio = df_ratio_2.merge(df_ratio, how = 'left', left_on = ['fiscalDateEnding','Ticker'], right_on = ['fiscalDateEnding','Ticker'])

/Users/YitingZhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [3]:
df_ratio

,fiscalDateEnding,Ticker,NPL,CAR,ROE,ROA,NIM
0,2022-12-31,JPM,0.003572,0.167985,0.150623,0.012012,0.022033
1,2022-12-31,MS,0.001146,0.193313,0.089314,0.007578,0.007859
2,2022-12-31,GS,0.014472,0.190552,0.045260,0.003679,0.005754
